# Setup

Import any required libraries.

In [ ]:
import bql
import plotly.express as px

In [ ]:
import plotly.io as pio

pio.renderers.default='plotly_mimetype+notebook'
pio.templates["iqmo"] = pio.templates["plotly"]
pio.templates["iqmo"].layout.margin = dict(l=50, r=50, t=50, b=50) 
pio.templates["iqmo"].layout.height = 250
pio.templates.default = "iqmo"

In [ ]:
bql_svc = bql.Service()

quarters = 10
index = 'SPX Index'

corr_query = f"""get(sales_rev_turn)
    for(members('{index}'))
    with(fpo=range(-{quarters-1}Q, 0Q), fpt=Q, fill=prev)
    preferences(addcols=all)"""
response = bql_svc.execute(corr_query)
spx_rev_df = bql.combined_df(response)


#### Pivot & Correlation Matrix

Pandas has a useful [`corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) function that computes the pairwise correlation between columns in a dataframe.

In [ ]:
# Pivot the data to wide: one column per security
df_pivoted = spx_rev_df.pivot_table(index='PERIOD_OFFSET', columns='ID', values='sales_rev_turn')

# Compute the correlation matrix
correlation_matrix = df_pivoted.corr()

# Show only 5 rows / 5 columns, for blog
display(correlation_matrix.head(5).iloc[:, :5])

In [ ]:
# Compute the period/period percent change of sales_rev_turn
spx_rev_df_sorted = spx_rev_df.sort_values(by=['ID','PERIOD'])

# Compute percent change
spx_rev_df_sorted['rev_pct_chg'] = spx_rev_df_sorted.groupby(level=0)['sales_rev_turn'].pct_change()

df_pivoted = spx_rev_df_sorted.pivot_table(index='PERIOD_OFFSET', columns='ID', values='rev_pct_chg')

correlation_matrix = df_pivoted.corr()
display(correlation_matrix.head(5).iloc[:, :5])

In [ ]:
small_matrix = correlation_matrix.head(5).iloc[:, :5]

px.imshow(small_matrix, text_auto=True)

#### DuckDB

The following is an example using DuckDB. DuckDB is a powerful in-memory analytics engine with a *lot* of features: it's great for problems that fit within a single server. 

I prefer the declarative syntax of SQL and BQL query strings to the various Pythonic APIs and DataFrame libraries. SQL queries can express complex, multi-"step" transformations in a single operation.

This is just to demonstrate **feasibility**. The Pandas example is certainly the more concise solution in this case. 

In [ ]:
# This conditional code is only for blogging purposes:
# To avoid reinstalling dependencies multiple times.
# Don't do this in real code. Load your dependencies in a Custom Env.

try:
    import duckdb
except Exception as e:
    print(f"Error importing, installing and trying again str({e})")
    %pip install duckdb
    import duckdb


In [ ]:
# Drop the pandas index, to make it easier to work with in duckdb.
spx_rev_df_c = spx_rev_df.reset_index()

df_corr = duckdb.sql("""
with q1 as 
    -- First, compute cross_correlation
    (SELECT a.id AS id1, 
            b.id AS id2, 
            CORR(a.sales_rev_turn, b.sales_rev_turn) AS correlation
        FROM spx_rev_df_c a
        JOIN spx_rev_df_c b ON a.period_offset = b.period_offset
        GROUP BY a.id, b.id
        ORDER BY a.id, b.id
    )
pivot q1 on id2 using last(correlation) group by id1 order by id1
""").df()

display(df_corr.set_index("id1").head(5).iloc[:, :5])